In [109]:
testing = '''
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14'''

In [196]:
testing=None

In [197]:
import numpy as np
scanners = {}
if testing:
    for line in testing.split('\n')[1:]:
        if ('---') in line:
            idx = int(line.strip()[12:].split(' ')[0])
            scanners[idx] = []
        elif line.strip() == '':
            continue
        else:
            x,y,z = [int(i) for i in line.strip().split(',')]
            scanners[idx].append((x,y,z))
else:
    with open('./input.txt','r') as file:
        for line in file:
            if ('---') in line:
                idx = int(line.strip()[12:].split(' ')[0])
                scanners[idx] = []
            elif line.strip() == '':
                continue
            else:
                x,y,z = [int(i) for i in line.strip().split(',')]
                scanners[idx].append((x,y,z))
for k in scanners.keys():
    scanners[k] = np.array(scanners[k])
    
def deg2rad(deg):
    return deg/360*2*np.pi

#make rotation matrix for given roll, pitch, and yaw
def rx(roll):
    roll = deg2rad(roll)
    Rx = np.array([[1, 0            , 0.          ],
                   [0, np.cos(roll), -np.sin(roll)],
                   [0, np.sin(roll),  np.cos(roll)]])
    return Rx

def ry(pitch):
    pitch = deg2rad(pitch)    
    Ry = np.array([[np.cos(pitch) , 0, np.sin(pitch)],
                   [0             , 1,             0],
                   [-np.sin(pitch), 0, np.cos(pitch)]])
    return Ry
    
def rz(yaw):
    yaw = deg2rad(yaw)    
    Rz = np.array([[np.cos(yaw), -np.sin(yaw), 0],
                   [np.sin(yaw),  np.cos(yaw), 0],
                   [0.         ,  0          , 1]])
    return Rz

def rotate():
    #do the first 4 faces (6,5,1,2)
    r = np.identity(3)    
    for _ in range(4):
        r = np.matmul(r,rx(90))
        for _ in range(4):
            r = np.matmul(r,rz(90)).astype(int)
            yield r
    #do face 3
    r = np.identity(3)
    r = np.matmul(r,ry(90))
    for _ in range(4):
        r = np.matmul(r,rz(90)).astype(int)
        yield r
    #do face 4        
    r = np.matmul(r,ry(180))    
    for _ in range(4):
        r = np.matmul(r,rz(90)).astype(int)
        yield r
        
def equal(a,b):
    counter = 0
    for x in a:
        for y in b:
            if np.all(x==y):
                counter +=1
    return counter
 
    
rotation = []
for r in rotate():
    rotation.append(r)
rotation = np.array(rotation)    

In [198]:
scan = scanners.copy()
temp = scan.copy()
for k,v in scan.items():
    temp[k] = np.matmul(rotation,v.T)
    temp[k] = np.swapaxes(temp[k],1,2) 

scan = temp.copy()
scan[0].shape

(24, 26, 3)

In [202]:
import pandas as pd
dd = []
for scanner in list(scan.keys()):
    for idx, s in enumerate(scan[scanner]):
        for x,y,z in s:
            dd.append({
                'rot':idx,
                'x':x,
                'y':y,
                'z':z,
                'scanner': scanner})
going = pd.DataFrame(dd)
everything = going.copy()
len(everything)/24

950.0

In [200]:
everything['scanner'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36])

506

In [217]:
weird = [34, 33, 31, 21, 35, 30, 29, 32, 27]

In [218]:
#everything = going.copy()
print(f"Starting with {len(everything)//24} beacons remaining from {len(everything['scanner'].unique())} scanners")
while len(everything['scanner'].unique())!=1:
    print('Wonder if we fail')
    winning = False
    looping1 = weird#everything['scanner'].unique()[::-1]
    for s1 in looping1:
        looping2 = weird[::-1]#everything['scanner'].unique()
        for s2 in looping2:
            if s1 == s2:
                continue

            yourCentre = everything[((everything['scanner']==s2) & (everything['rot']==0))][['x','y','z']]        
            for rot in everything['rot'].unique():

                toBeCentred = everything[((everything['scanner']==s1) & (everything['rot']==rot))][['x','y','z']]

                for centreBeacon1 in range(len(toBeCentred)):
                    for centreBeacon2 in range(len(yourCentre)):

                        altered = toBeCentred - toBeCentred.iloc[centreBeacon1] + yourCentre.iloc[centreBeacon2]
                        same = pd.merge(altered, yourCentre, on=['x','y','z'], how='outer', indicator=True)
                        
                        if len(same[(same['_merge']=='both')]) >= 12:
                            dd = []
                            for r in same[(same['_merge']=='left_only')][['x','y','z']].iloc():
                                newData = np.matmul(rotation,r)
                                newData = newData.T

                                for idx,nd in enumerate(newData):
                                    x,y,z=nd
                                    dd.append({
                                            'rot':idx,
                                            'x':x,
                                            'y':y,
                                            'z':z,
                                            'scanner': s2})

                            dd = pd.DataFrame(dd)

                            everything = pd.concat([everything, dd])
                            everything.reset_index(drop=True, inplace=True)
                            everything.drop(everything[(everything['scanner']==s1)].index, inplace=True)
                            everything.reset_index(drop=True, inplace=True)
                            winning = True
                            print(f"Beacons from scanner {s1} centering on {centreBeacon1} with rotation {rot} match Scanner {s2} centering on {centreBeacon2} - {len(everything)//24} beacons remaining from {len(everything['scanner'].unique())} scanners")
                            break
                    if winning:
                        break
                if winning:
                    break
            if winning:
                break
        if winning:
            break

Starting with 614 beacons remaining from 9 scanners
Wonder if we fail
Wonder if we fail
Wonder if we fail


KeyboardInterrupt: 

In [ ]:
everything[everything['rot']==0]

In [ ]:
15 scanners left

In [215]:
for scanner,gbdf in everything.groupby('scanner'):
    print(scanner, len(gbdf))

21 624
27 3624
29 2232
30 1896
31 936
32 2904
33 936
34 624
35 960


In [ ]:
518 beaconds is too high!!!

In [ ]:
[34, 33, 31, 21, 35, 30, 29, 32, 27]

13

In [ ]:
keys = list(scan.keys())
s1=scanners[0].copy()

while len(keys) > 1:
    keys = list(scan.keys())
    print(len(keys))
    winning = False
    for b1 in range(len(s1))[:-11]:
        for k2 in keys[1:]:
            for s in scan[k2]:
                for b2 in range(len(s)-11):
                    temp = s - s[b2] + s1[b1]
                    x = equal(s1,temp)
                    if x >= 12:
                        winning = True
                        break
                if winning:
                    break
            if winning:
                break
            #else:
            #    print(f'\tNot winning {b1} - {k2}')      
        if winning:
            break
    if winning:
        s1 = list(s1)
        for s in temp:
            if np.any(s == s1) == False:
                s1.append(s)
        s1 = np.array(s1)
        print(f'\tWinning {k2}')
        del scan[k2]                    

In [ ]:
def add_r(row):
    for i,r in enumerate(rot):
        row[f"r{i}"] = np.matmul(r,row['beacon']).astype(int)
    return row
    
def align_r(row, align_to):
    for a_idx, a in enumerate(align_to):
        for r_idx,r in enumerate(rot):
            row[f"a{a_idx}r{r_idx}"] = row[f"r{r_idx}"] + a
    return row
    
d = []
import pandas as pd
for k,v in scanners.items():
    for b in v:
        d.append({
            'scanner':k, 
            'beacon':b
        })
df = pd.DataFrame(d)


df = df.apply(lambda x: add_r(x), axis = 1)
align_to = df[(df['scanner']==0)]['beacon'].copy()

df = df.apply(lambda x: align_r(x,align_to), axis = 1)

for i,r in enumerate(rot):
    df = df.drop(labels=f'r{i}',axis=1)
df

In [ ]:
14+160+1400+160000

In [ ]:
merger = []
for i in df['scanner'].unique():#range(2,37):
    if i == 0:
        continue
    print(i)
    for j in range(26):
        for r in range(24):
            p = df[(df['scanner']==i)][['beacon',f'a{j}r{r}']]
            for q in range(len(p)):
                x = np.array([i for i in p[f'a{j}r{r}']])-np.array([i for i in p['beacon'].loc[p.index[q]]])
                y = pd.DataFrame([i for i in df[(df['scanner']==0)]['beacon'].to_numpy()])
                z = np.array([ i for i in pd.concat([y, pd.DataFrame(x)]).to_numpy()[:,0]])
                if z.size - np.unique(z,axis=0).size >= 12:
                    merger.append((i,j,r,q))
                    winning = True
                    break
            if winning:
                break
        if winning:
            break

In [ ]:
merger

In [ ]:
df['scanner'].unique()

In [ ]:
 s - s[b2] + s1[b1]